In [97]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [9]:
import json

### Utility 함수들

In [18]:
# utility functions

import json
def get_cost(index1, index2, cost_type, json_path, toll_include=True):
    """
    index1, index2와 cost_type을 지정해주면 json 파일을 파싱하여 비용을 반환
    """

    # 출발지와 도착지가 같은 경우 0 반환
    if index1 == index2:
        return 0
    else:
        from_idx = min(index1, index2)
        to_idx = max(index1, index2)

        # 마지막에 /가 없으면 붙이기
        if json_path[-1] != '/':
            json_path += '/'
        json_path = json_path + f'route_{from_idx}_{to_idx}.json'
        
        try:
            with open(json_path, 'r') as file:
                data = json.load(file)
        except:
            print(f"{json_path} 파일을 읽을 수 없음")
            raise ValueError()
        
        try:
            tollFare = 0 if toll_include else data['route'][cost_type][0]['summary']['tollFare']
            feulPrice = data['route'][cost_type][0]['summary']['fuelPrice']
            return tollFare+feulPrice
        except:
            print(f"{json_path} 파일의 tollFare와 fuelPrice 파싱 불가")
            raise ValueError()

### 수요 및 각 node와 depot의 cost

In [38]:
DEPOT_INDEX = 0
COST_TYPE = 'trafast'
mu = 300
std = 100

node_data = pd.read_csv('./../Data/data.csv')

# 데이터프레임 생성
node_demands = pd.DataFrame(columns=['node'], data=node_data[node_data['index'] != DEPOT_INDEX]['index'].values)

# 각 node에서의 수요(필요에 따라 코드 수정)
node_demands['demand'] = None
for node in node_demands['node']:
    machine_num = node_data.loc[node_data['index']==node, 'machine'].values[0]

    node_demand = 0

    # 기기수만큼 정규분포에서 랜덤 샘플링을 진행하여 더해줌
    while(machine_num != 0):
        node_demand += np.random.normal(mu, std)
        machine_num -= 1

    node_demand = round(node_demand)
    if node_demand < 0:
        node_demand = 0
    node_demands.loc[node_demands['node'] == node, 'demand'] = node_demand

# node와 depot의 cost
node_demands['depot_node_cost'] = node_demands['node'].apply(lambda x : get_cost(DEPOT_INDEX, x, COST_TYPE, './../Data/routes', toll_include=True))

In [41]:
node_demands.to_csv('./../Data/node_demand.csv', index=False)

### node간 cost matrix

In [108]:
DEPOT_INDEX = 0
COST_TYPE = 'trafast'
JSON_PATH = './../Data/routes'

node_data = pd.read_csv('./../Data/data.csv')
nodes = node_data[node_data['index']!=DEPOT_INDEX]['index'].values

# cost_matrix는 실제 node 두 개에 대해서는 유의미한 값, 아닌 node 조합에는 -1을 저장
cost_matrix = np.full((max(nodes)+1, max(nodes)+1), -1)

for i in tqdm(nodes):
    for j in nodes:
        if i == j:
            cost_matrix[i][i] = 0
            continue
        elif i > j:
            continue
        # i < j일 때에만 계산 후 반대쪽에도 채우기
        else:
            cost_matrix[i][j] = cost_matrix[j][i] = get_cost(i, j, COST_TYPE, JSON_PATH, toll_include=True)

100%|██████████| 53/53 [00:09<00:00,  5.35it/s]


In [109]:
# 배열을 파일로 저장
np.save('./../Data/cost_matrix.npy', cost_matrix)